# S2W5D6: ViT —— 给 Transformer 装上眼睛

## 1 核心理论：Patchify（切块化）

Transformer 的输入必须是 **序列 (Sequence)**，也就是一排向量 `[vector_1, vector_2, ...]`。但图片是一个**方块（Grid）**，也是一个像素矩阵`[H, W, C]`，我们需要将方块变成序列。

### 1.1 传统CNN的做法

一层层卷积，把图片变得越来越小，越来越厚。

### 1.2 ViT的做法（暴力美学）

像切蛋糕一样，把一张$224 \times 224$的大图，切成无数个$16 \times 16$的**小方块 (Patches)**。

- **每一个 Patch，就等于 NLP 里的一个单词 (Token)。**
- **计算**：
    - 图片大小: $224 \times 224$
    - Patch 大小: $16 \times 16$
    - 横着能切: $224 / 16 = 14$ 个
    - 竖着能切: $224 / 16 = 14$ 个
    - **Token 总数**: $14 \times 14 = 196$ 个。

这就是 ViT 的全部秘密：**Image $\to$ Patches $\to$ Linear Projection $\to$ Transformer Encoder**。

## 2 工程实践：用`Cconv2d`实现Patch Embedding

虽然逻辑上是“切块再拉直”，但在代码实现时，我们**绝对不会**写 `for` 循环去切图（太慢了）。最优雅的方法是使用 **卷积 (Convolution)**。

**原理**：如果你设置一个卷积核
- **Kernel Size** = Patch Size (比如 16)
- **Stride (步长)** = Patch Size (比如 16)

那么这个卷积核每走一步，就正好覆盖并处理了一个完整的 Patch，而且**互不重叠**。卷积的输出结果，直接就是这个 Patch 的 Embedding 向量！

**代码实现：**

In [1]:
import torch
import torch.nn as nn

class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        """
        Args:
            img_size：图片分辨率（224）
            patch_size：每个小方块的大小（16）
            in_chans：输入通道数（RGB图片是3）
            embed_dim：转换后的特征维度（768, 和BERT-Base一样）
        """
        super().__init__()
        self.img_size=img_size
        self.patch_size = patch_size

        # 计算 Patch 的总数量
        # (224 // 16) * (224 // 16) = 14 * 14 = 196
        self.num_patches = (img_size // patch_size) * (img_size // patch_size)

        # --- 核心魔法: 使用 Conv2d 做切片 ---
        # kernel_size=16, stride=16
        # 这意味着卷积核一次看 16x16 的区域，然后跳过 16 个像素看下一个
        # 输出通道 embed_dim 就是要把这个 Patch 压缩成的向量长度
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        """
        x: [Batch, Channels, Height, Width] (例如：[32, 3, 224, 224])
        """
        # 1. 卷积投影
        # input: [B, 3, 224, 224]
        # output: [B, 768, 14, 14]
        x = self.proj(x)

        # 2. 展平 (Flatten)
        # 我们需要序列，不需要 grid
        # 从维度 2 (H) 开始展平
        # output: [B, 768, 196] (196 = 14*14)
        x = x.flatten(2)

        # 3. 维度交换（Transpose）
        # Transformer 需要的输入是 [Batch, Seq_Len, Dim]
        # 目前是 [B, Dim, Seq_Len]，所以要交换 Dim 和 Seq_Len
        # output: [B, 196, 768]
        x = x.transpose(1, 2)

        return x

## 3 联调测试：接上昨天的Transformer

现在，我们要把这个 **PatchEmbedding** 和昨天写的 **TransformerEncoder** 连起来，组成一个完整的 **ViT** 原型。

In [3]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.transformer_components import TransformerEncoder

# 1. 模拟一张图片 batch
# [Batch=2, Channel=3, Height=224, Width=224]
dummy_img = torch.randn(2, 3, 224, 224)

# 2. 实例化 PatchEmbedding
patch_embed = PatchEmbedding(img_size=224, patch_size=16, in_chans=3, embed_dim=512)
# 注意：这里 embed_dim 设为 512，为了配合昨天写的 Encoder 默认参数

# 3. 运行 Patch Embedding
x = patch_embed(dummy_img)
print("Patch Embedding Output shape:", x.shape)
# 预期: [2, 196, 512]
# 解释: 2张图，每张图切成 196 个块，每个块是 512 维向量

# 4. 接上 Transformer Encoder (昨天写的)
# 假设昨天写的 Encoder 需要 vocab_size 参数，但在 ViT 里其实不需要查表了，
# 不过为了复用代码，我们可以暂时忽略 embedding 层，直接传给 encoder layers
# 或者更简单：我们仅仅把 x 传给 encoder 内部的 layers (模拟过程)

# 为了演示严谨性，我们只看维度匹配
print(f"这个 shape {x.shape} 完全符合 Transformer 的输入要求！")
print("它现在和 NLP 里的 [Batch, Seq_Len, Dim] 没有任何区别。")

# 如果你真的想跑通 ViT，只需要在 TransformerEncoder 的 forward 里，
# 跳过 self.embedding(src)，直接处理 dense vectors 即可。

Patch Embedding Output shape: torch.Size([2, 196, 512])
这个 shape torch.Size([2, 196, 512]) 完全符合 Transformer 的输入要求！
它现在和 NLP 里的 [Batch, Seq_Len, Dim] 没有任何区别。


## 4 深度解析 & 具身智能视角

### 🤖 具身智能面试题 (Embodied AI Interview)

**Q1: 为什么现在的机器人大模型（如 RT-2）都用 ViT 而不用 ResNet？**

  * **A**:
    1.  **统一模态 (Modality Alignment)**: 机器人的输入是复杂的（文本指令 + 图像观测）。如果用 CNN，图像出来的是 Feature Map，文本出来的是 Token，两者很难融合。如果用 ViT，**图像也是 Token，文本也是 Token**，它们可以直接拼接在一起：`[指令Token, 图片Token, 图片Token...]`，然后扔进同一个 Transformer 就能学会“看到图片执行指令”。
    2.  **全局注意力**: CNN 关注局部（卷积核），ViT 一上来就是 Global Attention。对于机器人来说，“桌子边缘的杯子”和“我的手”距离可能很远，ViT 能更好地捕捉这种远距离的空间关系。

**Q2: Patch Size 对性能有什么影响？**

  * **A**:
      * **Patch 越小 (如 8x8)**: Token 数量变多（$196 \to 784$），图像看得更细，但计算量激增（Attention 是序列长度的平方复杂度 $O(N^2)$）。
      * **Patch 越大 (如 32x32)**: Token 变少，计算快，但丢失细节，机器人可能看不清小螺丝钉。
      * **16x16**: 是目前工业界最常用的平衡点。

## 🚀 今日总结

恭喜！你刚刚打通了 **NLP** 和 **CV** 的任督二脉。

  * 你发现：**所谓的 CV 模型，只要做一步 `Conv2d` 切片，瞬间就变成了 NLP 序列问题。**
  * 这为你后续学习 **多模态大模型** 和 **具身 Agent** 扫清了最大的障碍。

**Stage 2 任务圆满完成！** 🎉

接下来，请花一点时间整理你的 `src` 文件夹。
明天，我们将进入 **Review Day (复盘日)**，也就是 **Week 6 的开始**。我们将基于 Hugging Face 库，用**工业界**的方式重写这一切，并开始训练 BERT 做文本分类。